In [1]:
# %%
# %% [markdown]
# # Integración de Pinecone con modelo Random Forest para etiquetado multilabel
#    usando en_core_web_sm (inglés)

# %%
# Instalar librerías necesarias (descomenta si hace falta)
# %pip install pinecone
# %pip install sentence-transformers
# %pip install joblib
# %pip install spacy

# %%
import os
import pandas as pd
import pinecone
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from pinecone import Pinecone, ServerlessSpec
import configparser
import time
from pinecone import exceptions
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as STOP_WORDS_EN  # <-- Usamos sólo inglés
import re

DEFAULT_BOOK_TITLE = "The Flea: The Amazing Story of Leo Messi"
DEFAULT_BOOK_BLURB = (
    "The captivating story of soccer legend Lionel Messi, from his first touch at age five in the streets of Rosario, Argentina, to his first goal on the Camp Nou pitch in Barcelona, Spain. The Flea tells the amazing story of a boy who was born to play the beautiful game and destined to become the world's greatest soccer player."
)

# %%
# 1. Leer credenciales de Pinecone desde config.cfg
config = configparser.ConfigParser()
config.read('../config.cfg')
PINECONE_API_KEY = config['pinecone']['api_key']
PINECONE_ENV = config['pinecone']['environment']

# %%
# 2. Inicializa Pinecone
try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    print("✅ Conectado a Pinecone!")
except Exception as e:
    print("❌ Error al conectar a Pinecone:", e)

index_name = "book-embeddings"  # Ajusta si lo necesitas

try:
    pc.create_index(
        name=index_name,
        dimension=384,    # Para un modelo SBERT típico
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print("✅ Índice creado correctamente.")
except exceptions.PineconeApiException as e:
    if "ALREADY_EXISTS" in str(e):
        print("📌 El índice ya existe. Usando el existente.")
    else:
        raise e

index = pc.Index(index_name)

# Borrar vectores en el namespace "books"
# try:
#     index.delete(delete_all=True, namespace="books")
#     print("🗑️ Vectores antiguos en 'books' borrados.")
# except pinecone.exceptions.NotFoundException:
#     print("ℹ️ Namespace 'books' no encontrado. Nada que borrar.")

# %%
# 3. Cargar modelos y binarizador
embedding_model = joblib.load("../model/book_tagging_pipeline_sentence_bert.joblib")
clf = joblib.load("../model/book_tagging_rf.joblib")
mlb = joblib.load("../model/book_tagging_rf_mlb.joblib")

# %%
# 4. Carga CSV y realiza transformaciones/renombrado como en tu ejemplo
books_df = pd.read_csv("../data/raw/goodreads_data.csv")

books_df.rename(columns={
    "Book": "book_title",
    "Description": "blurb",
    "Genres": "tags"
}, inplace=True)

books_df["tags"] = books_df["tags"].fillna("[]").apply(
    lambda x: ", ".join(
        tag.strip().lower().replace(" ", "-") for tag in eval(x)
    )
)

books_df = books_df.sample(1000, random_state=42)

books_df["book_title"] = books_df["book_title"].fillna("")
books_df["blurb"] = books_df["blurb"].fillna("")
books_df["tags"] = books_df["tags"].fillna("")
books_df["text"] = books_df["book_title"] + ". " + books_df["blurb"]

def parse_comma_tags(s: str):
    return [tag.strip() for tag in s.split(",") if tag.strip()]

books_df["list_tags"] = books_df["tags"].apply(parse_comma_tags)

print("Ejemplo de filas tras la limpieza y sample:")
print(books_df[["book_title", "tags"]].head(3))

# %%
# 5. Generar (o cargar) embeddings
if os.path.exists("../model/vectors.npy"):
    X_embeddings = np.load("../model/vectors.npy")
    print("📥 Embeddings cargados desde ../model/vectors.npy")
    if len(X_embeddings) != len(books_df):
        print("⚠️ El tamaño de vectors.npy no coincide con las 1000 filas actuales.")
else:
    X_embeddings = embedding_model.encode(books_df['text'].tolist(), show_progress_bar=True)
    np.save("../model/vectors.npy", X_embeddings)
    print("💾 Embeddings generados y guardados en ../model/vectors.npy")

# %%
# 6. Subir embeddings a Pinecone (en batches)
pinecone_data = []
for idx, (vec, tags) in enumerate(zip(X_embeddings, books_df["tags"])):
    if pd.isna(tags) or not isinstance(tags, str):
        tags = ''
    pinecone_data.append((str(idx), vec.tolist(), {"tags": tags}))

# batch_size = 1000
# for i in range(0, len(pinecone_data), batch_size):
#     batch = pinecone_data[i:i + batch_size]
#     index.upsert(vectors=batch, namespace="books")

# print("✅ Embeddings subidos a Pinecone.")

stats = index.describe_index_stats(namespace="books")
print(f"📊 Vectores cargados en el índice: {stats['total_vector_count']}")

# Prueba una query
query_vector = X_embeddings[0]
res = index.query(vector=query_vector.tolist(), top_k=3, include_metadata=True, namespace="books")
print(f"Ejemplo de respuesta Pinecone = {res}")

# %%
# 7. Carga spacy en inglés y define stopwords
nlp = spacy.load("en_core_web_sm")
STOPWORDS_COMBINADAS = STOP_WORDS_EN  # Solo las stopwords en inglés

# (Si quisieras filtrar verbos en inglés, podrías definir un regex distinto,
#  pero aquí lo eliminamos o lo dejamos vacío.)
VERBAL_SUFFIXES_EN = re.compile(r"(ing|ed|ly)$")  # Ejemplo (opcional)

# %%
# 8. Función de predicción con manejo de submodelos single-class
def predict_with_ensemble(title, blurb, top_k=5, threshold=0.3,
                          enrich_with_nouns=True, pinecone_top_tags=6):
    text = title + ". " + blurb
    embedding = embedding_model.encode([text])[0]

    # A) Probabilidades con Random Forest (MultiOutputClassifier)
    probs = []
    for estimator in clf.estimators_:
        if len(estimator.classes_) == 1:
            # Solo una clase
            if estimator.classes_[0] == 0:
                prob = 0.0
            else:
                prob = 1.0
        else:
            prob = estimator.predict_proba(embedding.reshape(1, -1))[0][1]
        probs.append(prob)

    probs = np.array(probs)
    pred_rf = (probs >= threshold).astype(int)
    pred_rf = np.array([pred_rf])
    tags_rf = list(mlb.inverse_transform(pred_rf)[0])

    # B) Pinecone: vecinos
    pinecone_result = index.query(
        vector=embedding.tolist(),
        top_k=top_k,
        include_metadata=True,
        namespace="books"
    )

    from collections import Counter
    pinecone_all_tags = []
    for match in pinecone_result.matches:
        if 'tags' in match.metadata and match.metadata['tags']:
            pinecone_all_tags += [tag.strip().lower() for tag in match.metadata['tags'].split(',')]

    pinecone_tag_counts = Counter(pinecone_all_tags)
    tags_pinecone = [tag for tag, _ in pinecone_tag_counts.most_common(pinecone_top_tags)]

    # C) Extraer sustantivos relevantes con Spacy (en inglés)
    tags_nouns = []
    if enrich_with_nouns:
        doc = nlp(text)
        tags_nouns = sorted(set(
            token.lemma_.lower()
            for token in doc
            if token.pos_ in ["NOUN", "PROPN"]
            # Filtramos verbos y auxiliares
            and token.pos_ not in ["VERB", "AUX"]
            # Eliminamos stopwords y tokens no alfanuméricos
            and token.lemma_.lower() not in STOPWORDS_COMBINADAS
            and token.is_alpha
            and len(token) > 3
        ))

    # D) Fusión
    fusion_final = sorted(set(
        tags_rf + tags_pinecone + tags_nouns
    ))

    return {
        "tags_rf": sorted(tags_rf),
        "tags_pinecone": tags_pinecone,
        "tags_nouns": tags_nouns,
        "tags_fusion": fusion_final
    }

# %%
# 9. Prueba con las variables por defecto
result = predict_with_ensemble(DEFAULT_BOOK_TITLE, DEFAULT_BOOK_BLURB)
print("Tags por Random Forest:", result["tags_rf"])
print("Tags por Pinecone:", result["tags_pinecone"])
print("Tags por Nouns:", result["tags_nouns"])
print("Tags combinados (fusión):", result["tags_fusion"])


✅ Conectado a Pinecone!
📌 El índice ya existe. Usando el existente.
Ejemplo de filas tras la limpieza y sample:
                                             book_title  \
6252                     El amor, las mujeres y la vida   
4684                                        The Lowland   
1731  I'll Be Gone in the Dark: One Woman's Obsessiv...   

                                                   tags  
6252  poetry, spanish-literature, romance, fiction, ...  
4684  fiction, india, historical-fiction, literary-f...  
1731  nonfiction, true-crime, audiobook, crime, myst...  
📥 Embeddings cargados desde ../model/vectors.npy
⚠️ El tamaño de vectors.npy no coincide con las 1000 filas actuales.
📊 Vectores cargados en el índice: 10000
Ejemplo de respuesta Pinecone = {'matches': [{'id': '0',
              'metadata': {'tags': 'classics, fiction, historical-fiction, '
                                   'school, literature, young-adult, '
                                   'historical'},
      